In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
data_set = pd.read_csv('../../data/TrueFX/EUR-USD/datos_procesados_OHLC_5T_EURUSD-2018-01.csv', index_col=0,
                      infer_datetime_format=True, parse_dates=True)

In [3]:
data_set.head()

,open,high,low,bid,ask
2018-01-02 00:00:00,1.20094,1.20094,1.20011,1.20015,1.20110
2018-01-02 00:05:00,1.20015,1.20062,1.20011,1.20047,1.20084
2018-01-02 00:10:00,1.20039,1.20098,1.20035,1.20082,1.20126
2018-01-02 00:15:00,1.20080,1.20097,1.20025,1.20035,1.20091
2018-01-02 00:20:00,1.20034,1.20057,1.20031,1.20049,1.20085


In [4]:
%run ../../code/PNLEstimatorWrapper.py
%run ../../code/EUtilities.py
EU = EUtilities()

In [5]:
window = 10
X, y_reg, bt = EU.build_dataset(data_set, window=window, bid_col='bid', binary_target=True, PNL=True)

In [6]:
X.head()

,0,1,2,3,4,5,6,7,8,9,PNL_0,PNL_1
2018-01-02 00:45:00,1.20015,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,-0.00007,-0.00030
2018-01-02 00:50:00,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,-0.00028,-0.00017
2018-01-02 00:55:00,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,1.20139,0.00071,-0.00103
2018-01-02 01:00:00,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,1.20139,1.20058,-0.00044,0.00017
2018-01-02 01:05:00,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,1.20139,1.20058,1.20085,-0.00011,-0.00015


In [7]:
X['open'] = data_set.filter(X.index,axis=0).open
X['high'] = data_set.filter(X.index,axis=0).high
X['low'] = data_set.filter(X.index,axis=0).low

In [8]:
X.head()

,0,1,2,3,4,5,6,7,8,9,PNL_0,PNL_1,open,high,low
2018-01-02 00:45:00,1.20015,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,-0.00007,-0.00030,1.20143,1.20150,1.20134
2018-01-02 00:50:00,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,-0.00028,-0.00017,1.20150,1.20150,1.20131
2018-01-02 00:55:00,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,1.20139,0.00071,-0.00103,1.20130,1.20142,1.20130
2018-01-02 01:00:00,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,1.20139,1.20058,-0.00044,0.00017,1.20138,1.20155,1.20022
2018-01-02 01:05:00,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,1.20139,1.20058,1.20085,-0.00011,-0.00015,1.20060,1.20100,1.20058


In [9]:
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, Activation, LSTM, Embedding

In [10]:
X.shape, bt.shape

((6229, 15), (6229,))

In [11]:
X1 = np.array(X[[col for col in X.columns if col!='PNL_0' and col!='PNL_1']])

In [12]:
scaler = MinMaxScaler(feature_range=(0, 1))
X1 = scaler.fit_transform(X1)

In [13]:
X1 = X1.reshape(X1.shape[0],X1.shape[1],1)
y = bt
X1.shape, y.shape

((6229, 13, 1), (6229,))

como es un modelo de multiclasificación toca utilizar categorical_crossentropy y para esta hay que pasar y en formato (n_filas,n_clases)

In [14]:
y1 = to_categorical(y,num_classes=4)

In [15]:
print(y1.shape)
y1

(6229, 4)


array([[ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.]])

In [16]:
X1.shape, y1.shape

((6229, 13, 1), (6229, 4))

In [229]:
model = Sequential()

model.add(LSTM(64, input_shape=(X1.shape[1],1)))
model.add(Dense(4, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### para cuadrar los pesos

In [230]:
sum(y==0),sum(y==1),sum(y==2),sum(y==3)

(2686, 2741, 400, 402)

In [231]:
sum(y==0)/sum(y==0), sum(y==0)/sum(y==1), sum(y==0)/sum(y==2), sum(y==0)/sum(y==3)

(1.0, 0.97993433053630064, 6.7149999999999999, 6.6815920398009947)

In [233]:
s_w = (y==0)*1 + (y==1)*0.97 + (y==2)*6.71 + (y==3)*6.68

In [242]:
model.fit(X1[:4000], y1[:4000], epochs=100, batch_size=32, class_weight={0:0.1,1:0.1,2:0.4,3:0.4},
         sample_weight=s_w[:4000])

Epoch 1/100
4000/4000 [==============================] - 2s - loss: 2.4402 - acc: 0.1775     
Epoch 2/100
4000/4000 [==============================] - 1s - loss: 2.4422 - acc: 0.1730     
Epoch 3/100
4000/4000 [==============================] - 2s - loss: 2.4532 - acc: 0.1578     
Epoch 4/100
4000/4000 [==============================] - 2s - loss: 2.4437 - acc: 0.1807     
Epoch 5/100
4000/4000 [==============================] - 2s - loss: 2.4426 - acc: 0.2033     
Epoch 6/100
4000/4000 [==============================] - 2s - loss: 2.4397 - acc: 0.1657     
Epoch 7/100
4000/4000 [==============================] - 2s - loss: 2.4412 - acc: 0.2065     
Epoch 8/100
4000/4000 [==============================] - 2s - loss: 2.4404 - acc: 0.1815     
Epoch 9/100
4000/4000 [==============================] - 2s - loss: 2.4423 - acc: 0.1792     
Epoch 10/100
4000/4000 [==============================] - 2s - loss: 2.4436 - acc: 0.1725     
Epoch 11/100
4000/4000 [==============================] - 2

4000/4000 [==============================] - 1s - loss: 2.4444 - acc: 0.1773     
Epoch 88/100
4000/4000 [==============================] - 1s - loss: 2.4450 - acc: 0.1140     
Epoch 89/100
4000/4000 [==============================] - 2s - loss: 2.4437 - acc: 0.1397     
Epoch 90/100
4000/4000 [==============================] - 1s - loss: 2.4418 - acc: 0.1557     
Epoch 91/100
4000/4000 [==============================] - 2s - loss: 2.4408 - acc: 0.1790     
Epoch 92/100
4000/4000 [==============================] - 1s - loss: 2.4457 - acc: 0.1785     
Epoch 93/100
4000/4000 [==============================] - 1s - loss: 2.4461 - acc: 0.1685     
Epoch 94/100
4000/4000 [==============================] - 2s - loss: 2.4443 - acc: 0.1613     
Epoch 95/100
4000/4000 [==============================] - 2s - loss: 2.4445 - acc: 0.1923     
Epoch 96/100
4000/4000 [==============================] - 2s - loss: 2.4430 - acc: 0.1625     
Epoch 97/100
4000/4000 [==============================] - 1s - 

In [237]:
pre = model.predict_classes(X1)

5984/6229 [===========================>..] - ETA: 0s

In [238]:
print('target 0: ', sum(pre==0))
print('target 1: ', sum(pre==1))
print('target 2: ', sum(pre==2))
print('target 3: ', sum(pre==3))

target 0:  49
target 1:  1207
target 2:  4398
target 3:  575


In [239]:
pnl_0 = X.PNL_0
pnl_1 = X.PNL_1

In [240]:
r = sum((pre==1)*pnl_1 + (pre==0)*pnl_0)

In [241]:
r

-0.028910000000004432

In [169]:
# %load ../../code/step_validation.py
def step_validationRNN(model, X, y, n_epochs, n_batch_size, class_weight, cv):
    '''
    Recibe el estimador,X,y, y un generador cv con el cual hace la validación
    dependiendo que la configuración que este tenga
    '''
    import pandas as pd
    import numpy as np
    result = []
    for index in cv:
        pnl_1 = X.iloc[index[1]].PNL_1
        pnl_0 = X.iloc[index[1]].PNL_0
        X_pre = np.array(X.iloc[index[1]][[col for col in X.columns if col!='PNL_0' and col!='PNL_1']])
        X_pre = X_pre.reshape(X_pre.shape[0],X_pre.shape[1],1)
        
        X1 = np.array(X.iloc[index[0]][[col for col in X.columns if col!='PNL_0' and col!='PNL_1']])
        X1 = X1.reshape(X1.shape[0],X1.shape[1],1)
        y1 = to_categorical(np.array(y[index[0]]))
        
        model.fit(X1, y1, epochs=n_epochs, batch_size=n_batch_size, class_weight=class_weight, verbose=0)
        pre = model.predict_classes(X_pre)
        r = sum((pre==1)*pnl_1 + (pre==0)*pnl_0)
        
        result.append(r)
    return np.array(result)

In [170]:
# %load ../../code/v_split.py
# Se crea un generador "v_split" para utilizar como método de validación cruzada
def v_splitRNN(X, n_bdtrain, n_bdtest, mday):

    """"
    Hace un particionado del dataset, para tomar n_bdtrain días para entrenar
    y n_bdtest para probar, además, mday representa el paso de día a correr.
    X, dataframe, se necesita el indice de este para separar por días.
    n_bdtrain, número de bussines day utilizados para train.
    n_bdtest, número de bussines day utilizados para test.
    mday, días a correr para cada validación.
    
    """
    import pandas as pd
    import numpy as np
    from datetime import date
    start_day = 0
    
    #Divide el data set según días de train, test y cuanto se va moviendo
    bussines_day = []
    dates = pd.unique(X.index.date) #saco las fechas para luego tomar solo año-mes-día

    for i in dates: bussines_day.append(date.__format__(i,'%Y-%m-%d')) #lista de los bussines day
    
    intervals = []
    count = 0
    for i in bussines_day:
        f = len(X[i])-1 +count
        intervals.append([count,f])
        count = f+1
    
    for i in range(len(intervals)-n_bdtrain):
        yield(np.arange(intervals[start_day:start_day+n_bdtrain][0][0],
                        intervals[start_day:start_day+n_bdtrain][n_bdtrain-1][1]+1),
              np.arange(intervals[n_bdtrain+start_day:n_bdtrain+start_day+n_bdtest][0][0],
                        intervals[n_bdtrain+start_day:n_bdtrain+start_day+n_bdtest][n_bdtest-1][1]+1))
    
        start_day += mday
        if start_day+n_bdtest > len(intervals)-n_bdtrain:
            break

In [171]:
model = Sequential()

model.add(LSTM(64, input_shape=(X1.shape[1],1), return_sequences=True))
model.add(Dense(4,activation='softmax'))
model.add(LSTM(32))
#model.add(Dense(10, activation="relu", input_shape=(10,)))
model.add(Dense(4, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [172]:
step_validationRNN(model,X,y,n_epochs=200, n_batch_size=32,class_weight={0:0.06,1:0.06,2:0.44,3:0.44}
                   ,cv=v_splitRNN(X,4,1,1))

23/23 [==============================] - 0sETA: 0s


array([ 0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
        0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
       -0.0049 , -0.01014, -0.01373, -0.02131, -0.01349, -0.01058,
       -0.00172, -0.01477, -0.00813, -0.01124,  0.     ])

In [173]:
np.mean(np.array([ 0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
        0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
       -0.0049 , -0.01014, -0.01373, -0.02131, -0.01349, -0.01058,
       -0.00172, -0.01477, -0.00813, -0.01124,  0.     ]))

-0.00478304347826087

In [105]:
for i in v_splitRNN(X,4,1,1):
    print(X.iloc[i[0]].shape, X.iloc[i[1]].shape)

(1119, 15) (1, 15)
(845, 15) (279, 15)
(840, 15) (283, 15)
(845, 15) (281, 15)
(844, 15) (282, 15)
(1125, 15) (280, 15)
(1126, 15) (1, 15)
(844, 15) (280, 15)
(843, 15) (282, 15)
(843, 15) (288, 15)
(851, 15) (282, 15)
(1132, 15) (283, 15)
(1135, 15) (1, 15)
(854, 15) (282, 15)
(848, 15) (280, 15)
(846, 15) (282, 15)
(845, 15) (286, 15)
(1130, 15) (284, 15)
(1132, 15) (1, 15)
(853, 15) (281, 15)
(852, 15) (281, 15)
(847, 15) (287, 15)
(850, 15) (23, 15)


In [34]:
# %load ../../code/v_split.py
# Se crea un generador "v_split" para utilizar como método de validación cruzada
def v_split(X, n_bdtrain, n_bdtest, mday):

    """"
    Hace un particionado del dataset, para tomar n_bdtrain días para entrenar
    y n_bdtest para probar, además, mday representa el paso de día a correr.
    X, dataframe, se necesita el indice de este para separar por días.
    n_bdtrain, número de bussines day utilizados para train.
    n_bdtest, número de bussines day utilizados para test.
    mday, días a correr para cada validación.
    
    """
    import pandas as pd
    import numpy as np
    from datetime import date
    start_day = 0
    
    #Divide el data set según días de train, test y cuanto se va moviendo
    bussines_day = []
    dates = pd.unique(X.index.date) #saco las fechas para luego tomar solo año-mes-día

    for i in dates: bussines_day.append(date.__format__(i,'%Y-%m-%d')) #lista de los bussines day
    
    intervals = []
    count = 0
    for i in bussines_day:
        f = len(X[i])-1 +count
        intervals.append([count,f])
        count = f+1
    
    for i in range(len(intervals)-n_bdtrain):
        yield(np.arange(intervals[start_day:start_day+n_bdtrain][0][0],
                        intervals[start_day:start_day+n_bdtrain][n_bdtrain-1][1]+1),
              np.arange(intervals[n_bdtrain+start_day:n_bdtrain+start_day+n_bdtest][0][0],
                        intervals[n_bdtrain+start_day:n_bdtrain+start_day+n_bdtest][n_bdtest-1][1]+1))
    
        start_day += mday
        if start_day+n_bdtest > len(intervals)-n_bdtrain:
            break

In [47]:
# %load ../../code/step_validation.py
def step_validationRNN(estimator, X, y, cv, n_epochs, n_batch_size):
    '''
    Recibe el estimador,X,y, y un generador cv con el cual hace la validación
    dependiendo que la configuración que este tenga
    '''
    import numpy as np
    result = []
    for index in cv:
        estimator.fit(X.iloc[index[0]], y[index[0]], epochs=n_epochs, batch_size=n_batch_size)
        result.append(estimator.score(X.iloc[index[1]], y[index[1]]))
    return np.array(result)